In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 


In [2]:
import nltk


In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [4]:
import re 
from textblob import TextBlob
from wordcloud import WordCloud

In [5]:
import cufflinks as cf

In [6]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [7]:
import warnings 
warnings.filterwarnings('ignore')
warnings.warn('this will not show')
pd.set_option('display.max_columns', None)

In [8]:
df =pd.read_csv(r'C:\Users\ADITYA JHA\Desktop\sentiment_analysis\amazon.csv')

In [9]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [10]:
df.shape

(4915, 12)

In [11]:
df.columns


Index(['Unnamed: 0', 'reviewerName', 'overall', 'reviewText', 'reviewTime',
       'day_diff', 'helpful_yes', 'helpful_no', 'total_vote',
       'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'],
      dtype='object')

In [12]:
df = df.sort_values('wilson_lower_bound', ascending = False)

In [13]:
df.drop('Unnamed: 0', inplace=True,axis=1)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [14]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum> 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum()/ df.shape[0]*100).sort_values(ascending=True)
    missing_df = pd.concat([n_niss,np.round(ratio_, 2)], axis=1, keys=['Misiing Values','Ratio'])
    missing_df = pd.dataframe(missing_df)
    return missing_df



In [16]:
def check_dataframe(df, head=5, tail=5):
    print('shape'.center(82,'~'))
    print('rows: {}'.format(df.shape[0]))
    print('columns: {}'.format(df.shape[1]))
    print('types'.center(82,'~'))
    print(df.dtypes)
    print(''.center(82,'~'))
    
check_dataframe(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~shape~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
rows: 4915
columns: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~types~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [17]:
def check_class(dataframe):
    ndf = pd.DataFrame({'Variable': dataframe.columns,
                                'Classes': [dataframe[i].nunique() \
                                            for i in dataframe.columns]})
    ndf = ndf.sort_values('Classes',ascending = False)
    ndf = ndf.reset_index(drop=True)
    return ndf
check_class(df)

,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [18]:
constraints = ['#B34D22', '#EBE00C', '#1FEB0C', '#0C92EB', '#EB0CD5']

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentage'),
                        specs=[[{'type': 'xy'}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=df[column_name].value_counts().values,
                         x=[str(i) for i in df[column_name].value_counts().index],
                         text=df[column_name].value_counts().values,
                         textfont=dict(size=14),
                         name=column_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=constraints,
                                     line=dict(color='#DBE6EC', width=1))),
                 row=1, col=1)

    fig.add_trace(go.Pie(labels=df[column_name].value_counts().index,
                          values=df[column_name].value_counts().values,
                          textfont=dict(size=20),
                          textposition='auto',
                          showlegend=False,
                          name=column_name,
                          marker=dict(colors=constraints)),
                 row=1, col=2)

    fig.update_layout(title={'text': column_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')
    fig.show()




In [19]:
categorical_variable_summary(df, 'overall')

In [20]:
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [21]:
review_example = df.reviewText[20]
review_example

'Works as expected.  High transfer speed.  Nice to have extra microSD adapters lying around since I always seem to misplace them.  Mainly bought it to use in my Nikon DSLR since I can put the card directly into my (CM modded) Nook HD+ tablet in the field and view my images on a larger screen.'

In [22]:
review_example = re.sub('[^a-zA-Z]',' ',review_example)
review_example

'Works as expected   High transfer speed   Nice to have extra microSD adapters lying around since I always seem to misplace them   Mainly bought it to use in my Nikon DSLR since I can put the card directly into my  CM modded  Nook HD  tablet in the field and view my images on a larger screen '

In [23]:
review_example = review_example.lower().split()
review_example

['works',
 'as',
 'expected',
 'high',
 'transfer',
 'speed',
 'nice',
 'to',
 'have',
 'extra',
 'microsd',
 'adapters',
 'lying',
 'around',
 'since',
 'i',
 'always',
 'seem',
 'to',
 'misplace',
 'them',
 'mainly',
 'bought',
 'it',
 'to',
 'use',
 'in',
 'my',
 'nikon',
 'dslr',
 'since',
 'i',
 'can',
 'put',
 'the',
 'card',
 'directly',
 'into',
 'my',
 'cm',
 'modded',
 'nook',
 'hd',
 'tablet',
 'in',
 'the',
 'field',
 'and',
 'view',
 'my',
 'images',
 'on',
 'a',
 'larger',
 'screen']

In [24]:
rt = lambda x: re.sub('[^a-zA-Z]',' ',str(x))
df['reviewText'] = df['reviewText'].map(rt)
df['reviewText'] = df['reviewText'].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob 
df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in df['reviewText'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    
    if neg > pos:
        df.loc[index, 'sentiment'] = 'Negative'
    elif pos > neg:
        df.loc[index, 'sentiment'] = 'Positive'
    else:
        df.loc[index, 'sentiment'] = 'Neutral'


In [26]:
df[df['sentiment']=='Positive'].sort_values('wilson_lower_bound',ascending=False).head(10)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive
1835,goconfigure,5,bought from bestbuy online the day it was anno...,28-02-2014,283,60,8,68,52,0.882353,0.784651,0.386111,0.616667,Positive
3981,"R. Sutton, Jr. ""RWSynergy""",5,the last few days i have been diligently shopp...,22-10-2012,777,112,27,139,85,0.805755,0.732136,0.070466,0.445756,Positive
4306,Stellar Eller,5,while i got this card as a deal of the day o...,06-09-2012,823,51,14,65,37,0.784615,0.670334,0.217172,0.503535,Positive
4596,"Tom Henriksen ""Doggy Diner""",1,hi i ordered two card and they arrived the nex...,22-09-2012,807,82,27,109,55,0.752294,0.663595,0.088242,0.452143,Positive
315,"Amazon Customer ""johncrea""",5,bought this card to use with my samsung galaxy...,13-08-2012,847,38,10,48,28,0.791667,0.657411,0.232500,0.555000,Positive


In [27]:
categorical_variable_summary(df,'sentiment')

In [28]:
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive


In [29]:
edf=df[['reviewText','sentiment']]
edf.head().reset_index()

,index,reviewText,sentiment
0,2031,update so my lovely wife boug...,Positive
1,3449,i have tested dozens of sdhc and micro sdhc ca...,Positive
2,4212,note please read the last update scroll to ...,Positive
3,317,if your card gets hot enough to be painful it...,Positive
4,4672,sandisk announcement of the first gb micro ...,Positive
